In [12]:
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
device = 'cuda:0'
train_MNIST = torchvision.datasets.MNIST('../../data', train=True)
test_MNIST = torchvision.datasets.MNIST('../../data', train=False)

In [ ]:
train_data = train_MNIST.train_data / 255
train_label = train_MNIST.train_labels
test_data = test_MNIST.test_data / 255
test_label = test_MNIST.test_labels

train_data = train_data.reshape(-1, 1, 28, 28)
test_data = test_data.reshape(-1, 1, 28, 28)

In [15]:
# (batch_size, channel, height, width)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1,),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.dense = nn.Sequential(
            nn.Linear(64*7*7, 1024),
            nn.Linear(1024, 128),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.reshape((x.size(0), -1))
        x = self.dense(x)
        return x

In [ ]:
learning_rate = 0 # to modify
batch_size = 0 # to modify
model = CNN().to(device)
optim = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

In [17]:
trainset = torch.utils.data.TensorDataset(train_data, train_label)
trainloader = torch.utils.data.DataLoader(
    dataset=trainset,
    batch_size=batch_size,
    shuffle=True,
    )

testset = torch.utils.data.TensorDataset(test_data, test_label)
testloader = torch.utils.data.DataLoader(
    dataset=testset,
    batch_size=batch_size,
    shuffle=True,
)

In [ ]:
epoch = 5
loss_fn = nn.CrossEntropyLoss()

for epoch_cnt in range(epoch):
    last_loss =  0
    
    for index, (data, target) in enumerate(trainloader):
        optim.zero_grad()
        data=data.to(device)
        target=target.to(device)
        prediction = model(data)
        loss = loss_fn(prediction, target)
        loss.backward()
        optim.step()

        last_loss=loss.data
    
    with torch.no_grad():
        cnt = 0
        for index, (input, output) in enumerate(testloader):
            input=input.to(device)
            output=output.to(device)
            prediction = model(input)
            prediction = torch.argmax(prediction, dim=1)
            cnt += torch.sum(prediction==output)
        print(f"acc : {round(int(cnt)/100,2)}%, loss : {last_loss}")

# you make acc over 99.0% ! (at last epoch)
    

In [25]:
# using matplotlib, show mnist_img & prediction

### code here ###